Notebook which adds necessary columns to "anupam_original.csv" - "relative_id", "point_group" and "crystal_system" and saves the result as template.csv

In [1]:
import json
import pandas as pd
import numpy as np

import sys
sys.path.append('..')
from src.band_plotters import DATA_DIRECTORY

In [2]:
df_anupam = pd.read_csv("../fingerprints/anupam_original.csv", index_col="ID")
df_anupam.head()

,formula,gen_formula,space_group,segments,flat_segments,flatness_score,discovery,binary_flatness,horz_flat_seg,exfoliation_eg,...,A,B,C,D,E,F,radio,f_orb,sg_sto_group,percentage_flat
ID,,,,,,,,,,,,,,,,,,,,,
2dm-1,IrF2,AB2,164,3,0,0.095102,bottom-up,0,0,0.234620,...,F,Ir,NaN,NaN,NaN,NaN,non-radioactive,no-f-in-valence,NaN,NaN
2dm-2,Ba2Sb,AB2,164,3,1,0.387410,bottom-up,0,0,0.210650,...,Ba,Sb,NaN,NaN,NaN,NaN,non-radioactive,no-f-in-valence,NaN,NaN
2dm-3,TlS,AB,2,4,4,0.846460,bottom-up,1,3,0.095794,...,S,Tl,NaN,NaN,NaN,NaN,non-radioactive,no-f-in-valence,276.0,24.2
2dm-4,MoCl2,AB2,166,5,4,0.713760,bottom-up,0,0,-0.055818,...,Cl,Mo,NaN,NaN,NaN,NaN,non-radioactive,no-f-in-valence,NaN,NaN
2dm-6,RuI2,AB2,164,3,1,0.264930,bottom-up,0,0,0.084831,...,I,Ru,NaN,NaN,NaN,NaN,non-radioactive,no-f-in-valence,NaN,NaN


In [3]:
df_anupam.columns

Index(['formula', 'gen_formula', 'space_group', 'segments', 'flat_segments',
       'flatness_score', 'discovery', 'binary_flatness', 'horz_flat_seg',
       'exfoliation_eg', 'decomp_eng', 'band_gap', 'point_group', 'crystal',
       'A', 'B', 'C', 'D', 'E', 'F', 'radio', 'f_orb', 'sg_sto_group',
       'percentage_flat'],
      dtype='object')

In [4]:
with open(DATA_DIRECTORY/'db.json') as f:
    print(json.loads(next(f)).keys())

dict_keys(['_id', 'sg_number', 'formula_pretty', 'nelements', 'creation_task_label', 'created_at', 'chemsys', 'material_id', 'discovery_process', 'formula_reduced_abc', 'sg_symbol', 'formula_anonymous', 'source_id', 'spacegroup', 'elements', '_tasksbuilder', 'structure', 'bandstructure', 'bandgap', 'thermo', 'energy_vdw_per_atom', 'magnetism', 'total_magnetization', 'calc_settings', 'energy_per_atom', 'relative_id', 'decomposition_energy', 'exfoliation_energy_per_atom'])


In [5]:
df = df_anupam.copy()
relative_id = pd.Series(index=df.index, dtype="object")
crystal_system = pd.Series(index=df.index, dtype="object")
point_group = pd.Series(index=df.index, dtype="object")


no_discovery_process = []

with open(DATA_DIRECTORY/'db.json') as f:
    for material in f:
        material=json.loads(material)
        mat_id = material["material_id"]
        
        try:
            relative_id[mat_id] = material["relative_id"]
        except:
            no_discovery_process.append(material["discovery_process"])

        crystal_system[mat_id] = material["spacegroup"]["crystal_system"]
        point_group[mat_id] = material["spacegroup"]["point_group"]
         
len(no_discovery_process), relative_id.isna().sum() # not equal because some materials in db.json aren't in Anupam's df

(161, 154)

In [7]:
df["relative_id"] = relative_id
df["crystal_system"] = crystal_system
df["point_group"] = point_group
df.head()

,formula,gen_formula,space_group,segments,flat_segments,flatness_score,discovery,binary_flatness,horz_flat_seg,exfoliation_eg,...,C,D,E,F,radio,f_orb,sg_sto_group,percentage_flat,relative_id,crystal_system
ID,,,,,,,,,,,,,,,,,,,,,
2dm-1,IrF2,AB2,164,3,0,0.095102,bottom-up,0,0,0.234620,...,NaN,NaN,NaN,NaN,non-radioactive,no-f-in-valence,NaN,NaN,2dm-4963,trigonal
2dm-2,Ba2Sb,AB2,164,3,1,0.387410,bottom-up,0,0,0.210650,...,NaN,NaN,NaN,NaN,non-radioactive,no-f-in-valence,NaN,NaN,2dm-3279,trigonal
2dm-3,TlS,AB,2,4,4,0.846460,bottom-up,1,3,0.095794,...,NaN,NaN,NaN,NaN,non-radioactive,no-f-in-valence,276.0,24.2,2dm-5155,triclinic
2dm-4,MoCl2,AB2,166,5,4,0.713760,bottom-up,0,0,-0.055818,...,NaN,NaN,NaN,NaN,non-radioactive,no-f-in-valence,NaN,NaN,2dm-4342,trigonal
2dm-6,RuI2,AB2,164,3,1,0.264930,bottom-up,0,0,0.084831,...,NaN,NaN,NaN,NaN,non-radioactive,no-f-in-valence,NaN,NaN,2dm-3574,trigonal


In [8]:
np.unique(df["point_group"]), np.unique(df["crystal_system"])

(array(['-1', '-3', '-3m', '-4', '-42m', '-6', '-6m2', '1', '2', '2/m',
        '222', '3', '32', '3m', '4', '4/m', '4/mmm', '422', '4mm', '6/mmm',
        '6mm', 'm', 'mm2', 'mmm'], dtype=object),
 array(['hexagonal', 'monoclinic', 'orthorhombic', 'tetragonal',
        'triclinic', 'trigonal'], dtype=object))

In [9]:
df.to_csv("../fingerprints/template.csv")